### Prediction gap test for regression= true_labels= true_labels= true_labels

### Imports

In [1]:
import pandas as pd
import os
import xgboost as xgb
from pathlib import Path
import json
from src.decision_tree.tree import parse_xgboost_dump,  TreeEnsemble
import numpy as np

In [2]:
while "notebooks" in os.getcwd():
    os.chdir("../")
os.chdir("src")

### Wine data

In [3]:
model_path = Path("../models")
model_name = "winequality_red"

Bias jest dodawany na koniej każdej predykcji, niestety nie zapisuje się w dumped.

In [4]:
trees = parse_xgboost_dump(model_path / f"{model_name}_dumped.txt")
some_trees = TreeEnsemble(trees.trees, model_path / f"{model_name}_saved.json")
#some_trees = TreeEnsemble(trees.trees[0:10])
df = pd.read_csv("../data/wine_quality/test_winequality_red_scaled.csv")
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         320 non-null    float32
 1   volatile_acidity      320 non-null    float32
 2   citric_acid           320 non-null    float32
 3   residual_sugar        320 non-null    float32
 4   chlorides             320 non-null    float32
 5   free_sulfur_dioxide   320 non-null    float32
 6   total_sulfur_dioxide  320 non-null    float32
 7   density               320 non-null    float32
 8   pH                    320 non-null    float32
 9   sulphates             320 non-null    float32
 10  alcohol               320 non-null    float32
 11  quality               320 non-null    int64  
dtypes: float32(11), int64(1)
memory usage: 16.4 KB


### Sprawdzenie eval() dla 20 punktów

In [5]:
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")

true_labels = df.iloc[:, -1]
adam_trees_eval = [some_trees.eval(df.iloc[i, :-1]) for i in range(len(df))]

x_dmatrix = xgb.DMatrix(df.iloc[:, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)

from numpy import exp

pd.DataFrame(zip(true_labels, adam_trees_eval, actual_xgb_model_eval, adam_trees_eval - actual_xgb_model_eval),
             columns=["Quality", "Eval by Adam Trees",
                      "Actual eval by XGB model", 'diff'])

/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/mateusz/Documents/tmp/prediction-gap/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a

,Quality,Eval by Adam Trees,Actual eval by XGB model,diff
0,7,5.682261,5.682260,9.536743e-07
1,5,5.603910,5.603909,4.768372e-07
2,6,5.591124,5.591124,0.000000e+00
3,6,5.468013,5.468012,9.536743e-07
4,7,5.642080,5.642080,-4.768372e-07
...,...,...,...,...
315,6,5.714418,5.714417,4.768372e-07
316,6,5.872979,5.872980,-4.768372e-07
317,7,5.713606,5.713605,9.536743e-07
318,5,5.560312,5.560310,1.907349e-06


~~Z jakiegoś powodu powstają małe różnice, które wraz ze wzrostem liczby drzew zaczynają się zwiększać i dochodzą do dosyć wyskich wartości jak na przykład 0.1~~ - rozwiązane
    

### Housing data

model_path = Path("../models")
model_name = "housing"
bias = float(get_bias(model_path, f"{model_name}_saved.json"))
trees = parse_xgboost_dump(model_path / f"{model_name}_dumped.txt")
some_trees = TreeEnsemble(trees.trees, bias)
df = pd.read_csv("../data/housing_data/test_housing_scaled.csv")
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")

true_labels = df.iloc[:20, -1]
adam_trees_eval = [some_trees.eval(df.iloc[i, :-1]) for i in range(20)]

x_dmatrix = xgb.DMatrix(df.iloc[:20, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)

from numpy import exp

pd.DataFrame(zip(true_labels, adam_trees_eval, actual_xgb_model_eval),
             columns=["Median price", "Eval by Adam Trees",
                      "Actual eval by XGB model"])